In [21]:
import pandas as pd
import os
import numpy as np
import nltk
import re
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.optim as optim
from google_drive_downloader import GoogleDriveDownloader as gdd
from torch.utils.data import DataLoader, Dataset
from sklearn.feature_extraction.text import CountVectorizer

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
df = pd.read_csv('/content/drive/MyDrive/df/twitter_sentiment_data.csv')
df

,sentiment,message,tweetid
0,-1,@tiniebeany climate change is an interesting h...,792927353886371840
1,1,RT @NatGeoChannel: Watch #BeforeTheFlood right...,793124211518832641
2,1,Fabulous! Leonardo #DiCaprio's film on #climat...,793124402388832256
3,1,RT @Mick_Fanning: Just watched this amazing do...,793124635873275904
4,2,"RT @cnalive: Pranita Biswasi, a Lutheran from ...",793125156185137153
...,...,...,...
43938,1,"Dear @realDonaldTrump,\nYeah right. Human Medi...",791307031919550464
43939,1,What will your respective parties do to preven...,791316857403936768
43940,2,RT @MikkiL: UN Poll Shows Climate Change Is th...,791357509101621249
43941,0,RT @taehbeingextra: i still can$q$t believe th...,791390042136641537


In [25]:
df.loc[(df['sentiment']==1),'Reason']='Man-Made'
df.loc[(df['sentiment']==0),'Reason']='Nutral'
df.loc[(df['sentiment']==-1),'Reason']='Negative'
df.loc[(df['sentiment']==2),'Reason']='News'
df.head(20)

,sentiment,message,tweetid,Reason
0,-1,@tiniebeany climate change is an interesting h...,792927353886371840,Negative
1,1,RT @NatGeoChannel: Watch #BeforeTheFlood right...,793124211518832641,Man-Made
2,1,Fabulous! Leonardo #DiCaprio's film on #climat...,793124402388832256,Man-Made
3,1,RT @Mick_Fanning: Just watched this amazing do...,793124635873275904,Man-Made
4,2,"RT @cnalive: Pranita Biswasi, a Lutheran from ...",793125156185137153,News
5,0,Unamshow awache kujinga na iko global warming ...,793125429418815489,Nutral
6,2,"RT @cnalive: Pranita Biswasi, a Lutheran from ...",793125430236684289,News
7,2,RT @CCIRiviera: Presidential Candidate #Donald...,793126558688878592,News
8,0,RT @AmericanIndian8: Leonardo DiCaprio's clima...,793127097854197761,Nutral
9,1,#BeforeTheFlood Watch #BeforeTheFlood right he...,793127346106753028,Man-Made


In [27]:
def remove_at_mentions(text):
    cleaned_text = re.sub(r'@\w+', '', text)
    return cleaned_text

In [28]:
#df=remove_at_mentions(df)
#for i in range (4):
#  print(df[i]['message'])
#df['message'] = df['message'].apply(re.sub(r'@\w+', '', text))
df['message'] = df['message'].apply(remove_at_mentions)
df.head()

,sentiment,message,tweetid,Reason
0,-1,climate change is an interesting hustle as it...,792927353886371840,Negative
1,1,"RT : Watch #BeforeTheFlood right here, as tra...",793124211518832641,Man-Made
2,1,Fabulous! Leonardo #DiCaprio's film on #climat...,793124402388832256,Man-Made
3,1,RT : Just watched this amazing documentary by ...,793124635873275904,Man-Made
4,2,"RT : Pranita Biswasi, a Lutheran from Odisha, ...",793125156185137153,News


In [29]:
df.iloc[6]['message']

'RT : Pranita Biswasi, a Lutheran from Odisha, gives testimony on effects of climate change &amp; natural disasters on the poÃ¢â‚¬Â¦'

In [30]:
print(tabulate(df.describe())

SyntaxError: ignored

In [32]:
df['message'] = df['message'].apply(lambda x: re.sub('[^a-zA-z0-9\s]', '', x.lower()))
df.head()

,sentiment,message,tweetid,Reason
0,-1,climate change is an interesting hustle as it...,792927353886371840,Negative
1,1,rt watch beforetheflood right here as travel...,793124211518832641,Man-Made
2,1,fabulous leonardo dicaprios film on climate ch...,793124402388832256,Man-Made
3,1,rt just watched this amazing documentary by l...,793124635873275904,Man-Made
4,2,rt pranita biswasi a lutheran from odisha giv...,793125156185137153,News


In [33]:
pattern = r'\d+'
df['message'] = df['message'].apply(lambda x: re.sub(pattern, '', x))
df.loc[:, ['message']]

,message
0,climate change is an interesting hustle as it...
1,rt watch beforetheflood right here as travel...
2,fabulous leonardo dicaprios film on climate ch...
3,rt just watched this amazing documentary by l...
4,rt pranita biswasi a lutheran from odisha giv...
...,...
43938,dear \nyeah right human mediated climate chang...
43939,what will your respective parties do to preven...
43940,rt un poll shows climate change is the lowest...
43941,rt i still canqt believe this gif of taehyung...


In [34]:
import string
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

In [35]:
df['message'] = df['message'].apply(remove_punctuation)
df.head()

,sentiment,message,tweetid,Reason
0,-1,climate change is an interesting hustle as it...,792927353886371840,Negative
1,1,rt watch beforetheflood right here as travel...,793124211518832641,Man-Made
2,1,fabulous leonardo dicaprios film on climate ch...,793124402388832256,Man-Made
3,1,rt just watched this amazing documentary by l...,793124635873275904,Man-Made
4,2,rt pranita biswasi a lutheran from odisha giv...,793125156185137153,News


In [36]:
import nltk
nltk.download('stopwords')
#Remove stopwords
stop=stopwords.words("english")
print(stop)
df.head(3)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,sentiment,message,tweetid,Reason
0,-1,climate change is an interesting hustle as it...,792927353886371840,Negative
1,1,rt watch beforetheflood right here as travel...,793124211518832641,Man-Made
2,1,fabulous leonardo dicaprios film on climate ch...,793124402388832256,Man-Made


In [37]:
stop.extend(["could","though","would","also","many",'much','rt','dear'])
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [38]:
nltk.download('punkt')
#After removing stopwords
df['message'] = df['message'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word not in stop]))
#df['Words'] = df['message	'].apply(lambda x : len(str(x).split()))

df.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,sentiment,message,tweetid,Reason
0,-1,climate change interesting hustle global warmi...,792927353886371840,Negative
1,1,watch beforetheflood right travels world tackl...,793124211518832641,Man-Made
2,1,fabulous leonardo dicaprios film climate chang...,793124402388832256,Man-Made
3,1,watched amazing documentary leonardodicaprio c...,793124635873275904,Man-Made
4,2,pranita biswasi lutheran odisha gives testimon...,793125156185137153,News


In [ ]:
df

,sentiment,message,tweetid
0,-1,climate change interesting hustle global warmi...,792927353886371840
1,1,watch beforetheflood right travels world tackl...,793124211518832641
2,1,fabulous leonardo dicaprios film climate chang...,793124402388832256
3,1,watched amazing documentary leonardodicaprio c...,793124635873275904
4,2,pranita biswasi lutheran odisha gives testimon...,793125156185137153
...,...,...,...
43938,1,yeah right human mediated climate change chine...,791307031919550464
43939,1,respective parties prevent climate change glob...,791316857403936768
43940,2,un poll shows climate change lowest global con...,791357509101621249
43941,0,still canqt believe gif taehyung saved human r...,791390042136641537


In [39]:
def remove_short_words(text):
    words = text.split()
    filtered_words = []
    for word in words:
        if len(word) > 2:
            filtered_words.append(word)
    filtered_text = " ".join(filtered_words)
    return filtered_text

In [40]:
df['message'] = df['message'].apply(remove_short_words)
df.head()

,sentiment,message,tweetid,Reason
0,-1,climate change interesting hustle global warmi...,792927353886371840,Negative
1,1,watch beforetheflood right travels world tackl...,793124211518832641,Man-Made
2,1,fabulous leonardo dicaprios film climate chang...,793124402388832256,Man-Made
3,1,watched amazing documentary leonardodicaprio c...,793124635873275904,Man-Made
4,2,pranita biswasi lutheran odisha gives testimon...,793125156185137153,News


In [41]:
corpus = df['message'].to_numpy()
corpus

array(['climate change interesting hustle global warming planet stopped warming yes suv boom',
       'watch beforetheflood right travels world tackle climate change httpstcolkdehjtnn htt',
       'fabulous leonardo dicaprios film climate change brilliant watch httpstcorvbrmxjw via',
       ...,
       'poll shows climate change lowest global concerns httpstcooopcqjvoga',
       'still canqt believe gif taehyung saved human race stopped global warming watered every plant',
       'wealthy fossil fuel industry know climate change real money make sure safe'],
      dtype=object)

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [43]:
X_train, X_test, y_train, y_test = train_test_split(df['message'], df['sentiment'], test_size=0.25, random_state=42)

In [44]:
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [45]:
logistic_regression = LogisticRegression()
svm = SVC()
rf = RandomForestClassifier()

In [46]:
logistic_regression.fit(X_train_tfidf, y_train)
y_pred = logistic_regression.predict(X_test_tfidf)

accuracy = accuracy_score(y_test, y_pred)
classification_report_log = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report_log)

Accuracy: 0.7080830147460404
Classification Report:
               precision    recall  f1-score   support

          -1       0.73      0.39      0.51       962
           0       0.62      0.40      0.49      1985
           1       0.71      0.89      0.79      5711
           2       0.75      0.66      0.70      2328

    accuracy                           0.71     10986
   macro avg       0.70      0.59      0.62     10986
weighted avg       0.70      0.71      0.69     10986



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [47]:
svm.fit(X_train_tfidf, y_train)
y_pred_svm = svm.predict(X_test_tfidf)

accuracy_svm = accuracy_score(y_test, y_pred_svm)
classification_report_svm = classification_report(y_test, y_pred_svm)

print("Accuracy:", accuracy_svm)
print("Classification Report:\n", classification_report_svm)

Accuracy: 0.7178226834152558
Classification Report:
               precision    recall  f1-score   support

          -1       0.78      0.38      0.51       962
           0       0.68      0.38      0.48      1985
           1       0.70      0.91      0.79      5711
           2       0.78      0.68      0.73      2328

    accuracy                           0.72     10986
   macro avg       0.74      0.59      0.63     10986
weighted avg       0.72      0.72      0.70     10986



In [48]:
#lstm model

class TextDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

In [49]:
def __len__(self):
        return len(self.data)

In [50]:
def __getitem__(self, index):
        return self.data[index], self.labels[index]

In [51]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

In [52]:
def forward(self, x):
        embedded = self.embedding(x)
        lstm_output, _ = self.lstm(embedded)
        logits = self.fc(lstm_output[:, -1, :])
        return logits

In [56]:
input_size = 43943
hidden_size = 128
num_classes = 4

In [57]:
train_data_tensor = torch.tensor(train_data)
train_labels_tensor = torch.tensor(train_labels)
test_data_tensor = torch.tensor(test_data)
test_labels_tensor = torch.tensor(test_labels)

NameError: ignored